In [ ]:
import re

In [ ]:
with open('app.py', 'r') as f:
    code =

In [ ]:
# Define the new cookie management code to insert
cookie_management_code = '''
# Cookie management for cross-session persistence
def get_cookie_manager():
    """Create and return a cookie manager instance"""
    return stx.CookieManager()

def save_user_id_to_cookie(cookie_manager, user_id):
    """Save user ID to a cookie"""
    cookie_manager.set("user_id", user_id, expires_at=datetime.now() + pd.Timedelta(days=30))

def load_user_id_from_cookie(cookie_manager):
    """Load user ID from cookie if it exists"""
    return cookie_manager.get("user_id")
'''

# Find the import section to add the streamlit-extras import
imports_pattern = r'import streamlit as st.*?(?=\n\n)'
updated_imports = re.search(imports_pattern, code, re.DOTALL).group(0)
updated_imports += '\nimport extra_streamlit_components as stx'

code = re.sub(imports_pattern, updated_imports, code, flags=re.DOTALL)

# Insert the cookie management code after ensure_data_dir_exists
pattern = r'def ensure_data_dir_exists\(\).*?ensure_data_dir_exists\(\)'
match = re.search(pattern, code, re.DOTALL)
if match:
    replacement = match.group(0) + '\n' + cookie_management_code
    code = code.replace(match.group(0

In [ ]:
# Update the get_user_id function to use cookies
updated_get_user_id = '''
def get_user_id():
    """Get a unique user ID or create one if it doesn't exist"""
    if "user_id" not in st.session_state:
        # Initialize cookie manager
        cookie_manager = get_cookie_manager()
        
        # Try to load from cookie first
        cookie_user_id = load_user_id_from_cookie(cookie_manager)
        
        if cookie_user_id:
            # Use ID from cookie
            st.session_state.user_id = cookie_user_id
            st.session_state.persistent_user_id = cookie_user_id
        elif "persistent_user_id" in st.session_state:
            # Use previously stored ID from session state
            st.session_state.user_id = st.session_state.persistent_user_id
        else:
            # Generate a new user ID
            st.session_state.user_id = str(uuid.uuid4())
            st.session_state.persistent_user_id = st.session_state.user_id
            
            # Save to cookie for persistence across sessions
            save_user_id_to_cookie(cookie_manager, st.session_state.user_id)
            
    return st.session_state.user_id
'''

# Replace the old get_user_id function
old_get_user_id_pattern = r'def get_user_id\(\).*?return st\.session_state\.user_id'
code = re.sub(old_get_user_id_pattern, updated_get_user_id, code, flags=

In [ ]:
# Update the page configuration section to initialize the cookie manager
page_config_pattern = r'st.set_page_config\(.*?\)'
match = re.search(page_config_pattern, code, re.DOTALL)
if match:
    updated_page_config = match.group(0) + '\n\n# Initialize cookie manager for persistent user ID\ncookie_manager = get_cookie_manager()'
    code = code.replace(match.group(0), updated_page_config)

In [ ]:
with open('app.py', 'w') as f:
    f.write(code)

print("Updated app.py with cross-session history support

In [ ]:
with open('requirements.txt', 'w') as f:
    f.write('streamlit\npandas\nextra-streamlit-components\npython-docx\n')

print("Created requirements.txt with the necessary dependencies.")